In [1]:
import numpy as np
import pandas as pd
from math import sqrt

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split,cross_val_score,cross_validate
from sklearn.model_selection import TimeSeriesSplit

from sklearn import linear_model
from sklearn.linear_model import Lasso,LassoCV,LassoLarsCV 

In [2]:
# Import datasets
returns = pd.read_excel("Returns_Clean.xlsx", index_col = 'Dates')
returns.index = pd.to_datetime(returns.index, format='%Y%m%d')

flows = pd.read_excel("Flows_Clean.xlsx", index_col = 'Dates')
flows.index = pd.to_datetime(flows.index, format='%Y%m%d')

returns_labeled = returns.copy()
returns_labeled.columns =['return_'+str(i) for i in returns.columns]
flows_labeled = flows.copy()
flows_labeled.columns = ['flow_'+str(i) for i in flows.columns]

print(returns_labeled.head())
print(flows_labeled.head())
print(len(returns))

            return_55976  return_41072  return_27748  return_75259  \
Dates                                                                
1993-04-01     -0.019685      0.019391     -0.007282     -0.003891   
1993-04-02     -0.020080      0.002717     -0.002445      0.058594   
1993-04-05     -0.024590     -0.006775     -0.002451      0.022140   
1993-04-06     -0.067227      0.009550     -0.007371      0.025271   
1993-04-07      0.000000      0.005405      0.000000      0.021127   

            return_40970  return_62519  return_21792  return_54084  \
Dates                                                                
1993-04-01      0.083333      0.027027      0.005348      0.020000   
1993-04-02     -0.038462     -0.026316     -0.007979     -0.019608   
1993-04-05     -0.040000      0.027027      0.005362      0.000000   
1993-04-06      0.041667      0.052632      0.005333      0.000000   
1993-04-07      0.040000     -0.050000      0.002653      0.000000   

            return

In [3]:
# Preprocessing the datasets

# the combine of returns and flows of stocks
S = pd.concat([returns, flows], axis = 1)
S_labeled = pd.concat([returns_labeled, flows_labeled], axis = 1)
# print(S.describe())

# Standardize the dataset
scaler = StandardScaler()
returns_scaled = pd.DataFrame(scaler.fit_transform(returns), columns =returns.columns)
flows_scaled = pd.DataFrame(scaler.fit_transform(flows), columns =flows.columns)
# print(returns_scaled.describe())
# print(flows_scaled.describe())

S_scaled = pd.concat([returns_scaled, flows_scaled], axis = 1)
# print(S_scaled.describe())

In [5]:
# Function that generates predictor matrix for individual stock i 
# inputs: stock i, r_i, f_i, and full datasets S. 
# outputs: 
# predictor matrix X_small, includes stock i’s own returns up to three lags and stock i’s net trading flows up to three lags
# precictor matrix X_full, full predictor variables matrix by adding lagged cross stock to X_small
# predictor matrix X_alter, full predictor variables not based on X_small
def predictor(i, r_i,f_i, S_labeled):
    #     predictor matrix X_small
    X_small = pd.DataFrame()   
    for j in range(1,4):
        r_t = pd.DataFrame(r_i.iloc[:len(r_i)-j])
        r_t.columns = [str(j) + '_lag return_' + str(i)]
        f_t = pd.DataFrame(f_i.iloc[:len(f_i)-j]) 
        f_t.columns = [str(j) + '_lag flow_' + str(i)]
        X_small = pd.concat([X_small, r_t], axis = 1)
        X_small = pd.concat([X_small, f_t], axis = 1)
    
    #     Add stock i’s lagged cross stock returns and flows to predictor matrix
    lagged_cross_stock = S_labeled.iloc[:len(r_i)-1]
    
    lagged_cross_stock.pop('return_'+str(i))
    lagged_cross_stock.pop('flow_'+str(i))
    
    #     generate full predictor variables matrix by adding lagged cross stock to X_small
    X_full = pd.concat([X_small, lagged_cross_stock], axis = 1)
    X_full.fillna(0, inplace = True)
    #     generate full predictor variables not based on X_small
    X_alter = pd.concat([S_labeled['return_'+str(i)].iloc[:len(r_i)-1], S_labeled['flow_'+str(i)].iloc[:len(r_i)-1]], axis = 1)
    X_alter = pd.concat([X_alter, lagged_cross_stock], axis = 1)
    X_alter.fillna(0, inplace = True)  

    return X_small, X_full, X_alter

In [6]:
# Visualize the Mean squre error on each fold 
# plot the mse & log(alphas) in LASSO model
# inputs: stock index and its LASSO model

def plot_mse(i,LA_model):
    
    # This is to avoid division by zero while doing np.log10
    EPSILON = 1e-4
    # Display results
    m_log_alphas = -np.log10(LA_model.alphas_ + EPSILON)
    # print(m_log_alphas)

    plt.figure()
    plt.plot(m_log_alphas, LA_model.mse_path_, ':')
    plt.plot(m_log_alphas, LA_model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
    plt.axvline(-np.log10(LA_model.alpha_ + EPSILON), linestyle='--', color='k',
            label='alpha: CV estimate')

    plt.legend()

    plt.xlabel('-log(alpha)')
    plt.ylabel('Mean square error')
    plt.title('Mean square error on each fold for stock ' + str(i))
    plt.axis('tight')

In [7]:

# Using first stock 55976 to demonstrate the process
l = returns.columns[1]
print(l)
#y1 = r_i_t is the variable to predict
y1 = returns[l].iloc[1:]
#predictor matrix X_small

r_l = pd.Series(returns_labeled['return_'+str(l)], index = returns.index)
f_l = pd.Series(flows_labeled['flow_'+str(l)], index = flows.index)
X1_small, X1_full, X1_alter = predictor(l, r_l, f_l, S_labeled)
# print(X1_full.iloc[:,1])
# split the training and test dataset using X_full
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_full, y1, test_size=0.2, random_state=0, shuffle = False)
# print(X1_test.shape)

#     Lasso model with cross-validation of 20-fold
LA_model = LassoCV(fit_intercept = False, max_iter = 100000, tol = 0.0001, cv = 20)

LA_model.fit(X1_train, y1_train)
LA_model.predict(X1_test)
l_score = pd.DataFrame([LA_model.score(X1_test, y1_test)], index = [l], columns = ['R squared'])
# print(LA_model.score(X1_test, y1_test))
# print(l_score)
# print(LA_model.coef_)
# print(LA_model.alpha_)
# print(LA_model.alphas_)


# plot_mse(l,LA_model)
coef_list = LA_model.coef_

features = pd.DataFrame()
LA_feature = []
for i in range(len(coef_list)):
    coef_x = X1_full.iloc[:,i]
        
    if coef_list[i] != 0:
        features = pd.concat([features, coef_x], axis = 1)
if features.empty:
    features = X_full

LA_feature.append(features)


41072


In [8]:
LASSO_models = []
LASSO_scores = []
LASSO_features = []
LA_mses = []
LA_rmse = []
# print(X.iloc[:len(returns)-1])
for i in tqdm(returns.columns):
#     y = r_i_t is the variable to predict
    y = returns[i].iloc[1:]
#     predictor matrix X_small, includes stock i’s own returns up to three lags and stock i’s net trading flows up to three lags
    r_i = pd.Series(returns_labeled['return_'+str(i)], index = returns.index)
    f_i = pd.Series(flows_labeled['flow_'+str(i)], index = flows.index)
    X_small, X_full, X_alter = predictor(i, r_i, f_i, S_labeled)
    
#   split the training and test dataset using X_full
    X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=0, shuffle = False)
    # print(X_test.shape)

#     Lasso model using cross-validation of 20-fold
    LA_model = LassoCV(fit_intercept = False, max_iter = 100000, tol = 0.0001, cv = 20)
    LA_model.fit(X_train, y_train)

# Find significant coefficient as features for further algorithms
    coef_list = LA_model.coef_

    features = pd.DataFrame()
    for i in range(len(coef_list)):
        coef_x = X_full.iloc[:,i]
#         find coefficients that is different from 0 
        if coef_list[i] != 0:
#         non-zero cofficients of variables are considered as predictors for RF 
            features = pd.concat([features, coef_x], axis = 1)
#     print(features)
# if coeffients are all zero, we use X_full as the predictor matrix
    if features.empty:
        features = X_full
    LASSO_features.append(features)
        
#     LA_model.predict(X_test)
    score = {i : LA_model.score(X_test, y_test)}
    LASSO_scores.append(score)
    LASSO_models.append(LA_model)
    y_pred = LA_model.predict(X_test)
    mse = {i : mean_squared_error(y_pred, y_test)}
    LA_mses.append(mse)
    rmse = {i : sqrt(mean_squared_error(y_pred, y_test))}
    LA_rmse.append(rmse)

In [9]:
LASSO_alphas = [model.alpha_ for model in LASSO_models]
LASSO_coefs = [model.coef_ for model in LASSO_models]
LASSO_intercepts = [model.intercept_ for model in LASSO_models]
# print(LASSO_coefs[1][203])
print(type(LASSO_features[0]))
LASSO_features[0].head()


<class 'pandas.core.frame.DataFrame'>


,1_lag return_55976,1_lag flow_55976,2_lag return_55976,2_lag flow_55976,3_lag return_55976,3_lag flow_55976,return_41072,return_27748,return_75259,return_40970,...,flow_63829,flow_44986,flow_11891,flow_22752,flow_27780,flow_59379,flow_75162,flow_27705,flow_54594,flow_48725
Dates,,,,,,,,,,,,,,,,,,,,,
1993-04-01,-0.019685,-4.696503,-0.019685,-4.696503,-0.019685,-4.696503,0.019391,-0.007282,-0.003891,0.083333,...,-3.762768,-1.733699,1.126364,-7.138398,3.073116,1.617669,2.448601,3.493056,0.902557,-0.535050
1993-04-02,-0.020080,-6.975709,-0.020080,-6.975709,-0.020080,-6.975709,0.002717,-0.002445,0.058594,-0.038462,...,-0.419981,-2.612104,0.019097,-7.138398,2.033127,-2.418096,-1.523983,-9.594576,-10.226385,-0.274176
1993-04-05,-0.024590,-5.642141,-0.024590,-5.642141,-0.024590,-5.642141,-0.006775,-0.002451,0.022140,-0.040000,...,-0.474352,1.625321,1.149879,-6.056788,2.309195,4.493038,0.854999,2.615968,0.817579,1.478560
1993-04-06,-0.067227,-6.975709,-0.067227,-6.975709,-0.067227,-6.975709,0.009550,-0.007371,0.025271,0.041667,...,-1.294707,1.255646,0.803975,-2.927825,-5.331582,1.635919,-9.876419,-2.862576,-3.755393,1.674835
1993-04-07,0.000000,-6.975709,0.000000,-6.975709,0.000000,-6.975709,0.005405,0.000000,0.021127,0.040000,...,0.000000,8.642735,1.060310,-2.069588,-1.625845,2.295149,0.570760,5.847429,-1.580178,1.814051


In [11]:
LASSO_features[0].shape

(1957, 204)

In [12]:
LASSO_features[0].to_csv('selected_features_X.csv')